In [13]:
import ROOT as rt
import sys
import numpy as np
import matplotlib.pyplot as plt


In [14]:
rt.gStyle.SetOptStat(0)
# camerakey=sys.argv[1]
camerakey="C1"
print("camera: ",camerakey)

indir="/Users/kunori/skdir/MuonX/mcAnalysis/icsim_02_01_ana/ntupleDir/"
infiles={"C0RAA":"all_C0RAA_012.root","C0RRR":"all_C0RRR_012.root",
         "C1RAA":"all_C1RAA_1314.root","C1RRR":"all_C1RRR_1314.root",
         "C2RAA":"all_C2RAA_012.root","C2RRR":"all_C2RRR_012.root",
         "C3RAA":"all_C3RAA_012.root","C3RRR":"all_C3RRR_012.root",
         }


outhistname="outhist_"+camerakey+".root"
fout=rt.TFile(outhistname,"recreate")

camera:  C1


In [15]:
def readYZ(key):
   f=rt.TFile(indir+infiles[key])

   histCryTime=f.Get("all_CryTime")
   days=(histCryTime.GetBinContent(1)/3600./24.)*1.0
   daysStr=str(round(days,1))

   tree = rt.gROOT.FindObject('tree')

   fout.cd()
   hname=key+"yz"
   h2_yz = rt.TH2D(hname,hname,20,-10.0,10.0,16,0.0,8.0)

   xyMaxEvents=0    # limit events to process for debug. 0 for all events, 
   print(" ",key,"    nEntries: ",tree.GetEntries(),"    xyMaxEvents: ",xyMaxEvents)
   n=0
   for ev in tree:
      n=n+1
      if n > xyMaxEvents and xyMaxEvents >0:
          break

      if ev.nHitRef[0] == 0:
         continue
      h2_yz.Fill(ev.RefProjVy[0],ev.RefProjVz[0])

   print("key=",key,"   type(h2_yz)=",type(h2_yz))

   f.Close()
   return h2_yz,daysStr
    
    

In [16]:
def makeYZplot(key):
   raakey=camerakey+"RAA"
   h2RAA, days1=readYZ(raakey)
   rrrkey=camerakey+"RRR"
   h2RRR, days2=readYZ(rrrkey)

   h2RAA.Divide(h2RRR)
   htitle=camerakey+"-view"+"     "+days1+" days"

   c1 = rt.TCanvas( 'c1', 'The Fit Canvas', 200, 10, 700, 500 )
   c1.cd()

   h2RAA.Draw("COLZ")
   h2RAA.SetAxisRange(0.8, 1.2,"Z")
   h2RAA.SetTitle(htitle)
   h2RAA.GetXaxis().SetTitle("Y (m)")
   h2RAA.GetYaxis().SetTitle("Z (m)")
   pdffile="plot_yz_"+camerakey+".pdf"
   c1.Print(pdffile)

In [17]:
def makeZ123ratio(zkey):
   yband={"Z1":"Y=[-6,-2]","Z2":"Y=[-2,2]","Z3":"Y=[+2,+6]"}   

   raakey=camerakey+"RAA"
   rrrkey=camerakey+"RRR"
   fa=rt.TFile(indir+infiles[raakey])  
   fb=rt.TFile(indir+infiles[rrrkey])

   histCryTime=fa.Get("all_CryTime")
   days=(histCryTime.GetBinContent(1)/3600./24.)*1.0
   daysStr=str(round(days,1))

   hname="Proj"+zkey

   histA=fa.Get(hname)
   histB=fb.Get(hname)
   histA.Divide(histB)

   htitle=camerakey+" Projected-Z "+yband[zkey]+"     "+daysStr+" days"

   c2 = rt.TCanvas( 'c2', 'The Fit Canvas', 200, 10, 700, 500 )
   c2.cd()
   c2.SetGrid()

   histA.Draw()
   histA.SetAxisRange(0.0, 8.0,"X")
   histA.SetAxisRange(0.8, 1.2,"Y")
   histA.SetTitle(htitle)
   histA.GetXaxis().SetTitle("Z (m)")
   pdffile="Zratio_"+camerakey+"_"+zkey+".pdf"
   c2.Print(pdffile)



In [18]:
makeYZplot(camerakey) 

makeZ123ratio("Z1")
makeZ123ratio("Z2")
makeZ123ratio("Z3")

  C1RAA     nEntries:  0     xyMaxEvents:  0
key= C1RAA    type(h2_yz)= <class cppyy.gbl.TH2D at 0x7f910ede0610>
  C1RRR     nEntries:  0     xyMaxEvents:  0
key= C1RRR    type(h2_yz)= <class cppyy.gbl.TH2D at 0x7f910ede0610>


Warning in <TCanvas::Constructor>: Deleting canvas with same name: c1
Info in <TCanvas::Print>: pdf file plot_yz_C1.pdf has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c2
Info in <TCanvas::Print>: pdf file Zratio_C1_Z1.pdf has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c2
Info in <TCanvas::Print>: pdf file Zratio_C1_Z2.pdf has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c2
Info in <TCanvas::Print>: pdf file Zratio_C1_Z3.pdf has been created


In [24]:
def makeZ123ratioErrorBars(zkey):
   yband={"Z1":"Y=[-6,-2]","Z2":"Y=[-2,2]","Z3":"Y=[+2,+6]"}   

   raakey=camerakey+"RAA"
   rrrkey=camerakey+"RRR"
   fa=rt.TFile(indir+infiles[raakey])  
   fb=rt.TFile(indir+infiles[rrrkey])

   histCryTime=fa.Get("all_CryTime")
   days=(histCryTime.GetBinContent(1)/3600./24.)*1.0
   daysStr=str(round(days,1))

   hname="Proj"+zkey

   histA=fa.Get(hname)
   histB=fb.Get(hname)

   pEff=rt.TEfficiency(histA,histB)

   # histA.Divide(histB)

   htitle=camerakey+" Projected-Z "+yband[zkey]+"     "+daysStr+" days"

   c2 = rt.TCanvas( 'c2', 'The Fit Canvas', 200, 10, 700, 500 )
   c2.cd()
   c2.SetGrid()

   pEff.Draw()
   # pEff.SetAxisRange(0.0, 8.0,"X")
   # Eff.SetAxisRange(0.8, 1.2,"Y")
   # pEff.SetTitle(htitle)
   # pEff.GetXaxis().SetTitle("Z (m)")
   pdffile="ZratioWerror_"+camerakey+"_"+zkey+".pdf"
   c2.Print(pdffile)



In [25]:
makeZ123ratioErrorBars("Z2")

Info in <TROOT::TEfficiency::CheckEntries>: Histograms are not consistent: passed bin content > total bin content
Error in <TROOT::TEfficiency::CheckConsistency>: passed TEfficiency objects do not have consistent bin contents
Error in <TEfficiency::TEfficiency(const TH1&,const TH1&)>: histograms are not consistent -> results are useless
Warning in <TEfficiency::TEfficiency(const TH1&,const TH1&)>: using two empty TH1D('h1','h1',10,0,10)
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c2
Info in <TCanvas::Print>: pdf file ZratioWerror_C1_Z2.pdf has been created
